<a href="https://colab.research.google.com/github/PatrickRuan/ML100Days/blob/master/homework/Day_025_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察均值編碼的效果

# [作業重點]
- 仿造範例, 完成標籤編碼與均值編碼搭配邏輯斯迴歸的預測
- 觀察標籤編碼與均值編碼在特徵數量 / 邏輯斯迴歸分數 / 邏輯斯迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 

# 作業1
* 請仿照範例，將鐵達尼範例中的類別型特徵改用均值編碼實作一次

In [2]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

import os
from google.colab import drive
drive.mount('/content/drive/')
!cp -r 'drive/My Drive/學習/Cupoy/ML100_5/d007_data' ./
os.chdir('/content/d007_data')
df_train = pd.read_csv('titanic_train.csv')
df_test = pd.read_csv('titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
cp: cannot stat 'drive/My Drive/學習/Cupoy/ML100_5/d007_data': No such file or directory


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 觀察鐵達尼生存預測中，均值編碼與標籤編碼兩者比較，哪一個效果比較好? 可能的原因是什麼?

In [10]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for i in df.columns:
  df_temp[i] = LabelEncoder().fit_transform(df[i])
start = time.time()

print(cross_val_score(LogisticRegression(), df_temp[:train_num], train_Y, cv=5).mean(), time.time()-start)


0.7800138095537004 0.12600159645080566


In [51]:
# https://ithelp.ithome.com.tw/articles/10194006
data = pd.concat([df[:train_num], train_Y], axis=1)

#print(data.columns)
#print(col)
col = 'Sex'
temp= data.groupby([col])['Survived'].mean().reset_index()
# groupby 形成的 dataframe長得都有點奇怪，左上角會有一個包含 groupby item 的 INDEX, 需要用 RESET_INDEX() 來處理
print(temp,'\n')

temp.columns = [col, f'{col}_mean']
print(temp,'\n')

data = pd.merge(data, temp, on=col, how='left')
print(data[[col,f'{col}_mean', 'Survived']])

data = data.drop([col] , axis=1)




      Sex  Survived
0  female  0.742038
1    male  0.188908 

      Sex  Sex_mean
0  female  0.742038
1    male  0.188908 

        Sex  Sex_mean  Survived
0      male  0.188908         0
1    female  0.742038         1
2    female  0.742038         1
3    female  0.742038         1
4      male  0.188908         0
..      ...       ...       ...
886    male  0.188908         0
887  female  0.742038         1
888  female  0.742038         0
889    male  0.188908         1
890    male  0.188908         0

[891 rows x 3 columns]


In [56]:
# 如果欄位類別數太多, 需要先剔除, 否則會 overfit (正確率會到達 1.0) # 哈，有想到
train_X.nunique()

Name        891
Sex           2
Ticket      681
Cabin       148
Embarked      4
dtype: int64

In [59]:
# 均值編碼 + 邏輯斯迴歸

data = pd.concat([df[:train_num], train_Y], axis=1)


for c in df.columns:
    mean_df = data.groupby([c])['Survived'].mean().reset_index()
    mean_df.columns = [c, f'{c}_mean']
    data = pd.merge(data, mean_df, on=c, how='left')
    data = data.drop([c] , axis=1)
data = data.drop(['Survived',  'Name_mean', 'Ticket_mean'] , axis=1) # 如果不 drop "Name_mean", 'Ticket_mean' 會太強

estimator = LogisticRegression()
start = time.time()
train_X = df_temp[:train_num]

print(f'shape : {df_temp[:train_num].shape}')
print(f'score : {cross_val_score(estimator, data, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')

shape : (891, 5)
score : 0.7867365513778168
time : 0.05696439743041992 sec
